In [1]:
from marubatsu import Marubatsu

def __init__(self, board_size=3, count_linemark=False):
    # ゲーム盤の縦横のサイズ
    self.BOARD_SIZE = board_size
    # 各直線上のマークの数を数えるかどうか
    self.count_linemark = count_linemark
    # 〇×ゲーム盤を再起動するメソッドを呼び出す
    self.restart()
    
Marubatsu.__init__ = __init__

In [2]:
def restart(self):
    self.initialize_board()
    self.turn = Marubatsu.CIRCLE     
    self.move_count = 0
    self.status = Marubatsu.PLAYING
    self.last_move = -1, -1          
    self.last_turn = None
    self.records = [self.last_move]
    if self.count_linemark:
        self.rowcount = {
            Marubatsu.CIRCLE: [0] * self.BOARD_SIZE,
            Marubatsu.CROSS: [0] * self.BOARD_SIZE,
        }
        self.colcount = {
            Marubatsu.CIRCLE: [0] * self.BOARD_SIZE,
            Marubatsu.CROSS: [0] * self.BOARD_SIZE,
        }
        self.diacount = {
            Marubatsu.CIRCLE: [0] * 2,
            Marubatsu.CROSS: [0] * 2,
        }
    
Marubatsu.restart = restart

In [3]:
def move(self, x, y):
    if self.place_mark(x, y, self.turn):
        self.last_turn = self.turn
        self.turn = Marubatsu.CROSS if self.turn == Marubatsu.CIRCLE else Marubatsu.CIRCLE  
        self.move_count += 1
        self.last_move = x, y
        if self.count_linemark:
            self.colcount[self.last_turn][x] += 1
            self.rowcount[self.last_turn][y] += 1
            if x == y:
                self.diacount[self.last_turn][0] += 1        
            if x + y == self.BOARD_SIZE - 1:
                self.diacount[self.last_turn][1] += 1        
        self.status = self.judge()
        if len(self.records) <= self.move_count:            
            self.records.append(self.last_move)
        else:
            self.records[self.move_count] = self.last_move
            self.records = self.records[0:self.move_count + 1]
            
Marubatsu.move = move

In [4]:
def unmove(self):
    if self.move_count > 0:
        x, y = self.last_move
        if self.count_linemark:
            self.colcount[self.last_turn][x] -= 1
            self.rowcount[self.last_turn][y] -= 1
            if x == y:
                self.diacount[self.last_turn][0] -= 1        
            if x + y == self.BOARD_SIZE - 1:
                self.diacount[self.last_turn][1] -= 1           
        if self.move_count == 0:
            self.last_move = (-1, -1)
        self.move_count -= 1
        self.turn, self.last_turn = self.last_turn, self.turn
        self.status = Marubatsu.PLAYING
        x, y = self.records.pop()
        self.remove_mark(x, y)
        
        self.last_move = self.records[-1]  
        
Marubatsu.unmove = unmove

In [5]:
def is_winner(self, player):
    x, y = self.last_move
    if self.count_linemark:
        if self.rowcount[player][y] == self.BOARD_SIZE or \
        self.colcount[player][x] == self.BOARD_SIZE:
            return True
        # 左上から右下方向の判定
        if x == y and self.diacount[player][0] == self.BOARD_SIZE:
            return True
        # 右上から左下方向の判定
        if x + y == self.BOARD_SIZE - 1 and \
            self.diacount[player][1] == self.BOARD_SIZE:
            return True
    else:
        if self.is_same(player, coord=[0, y], dx=1, dy=0) or \
        self.is_same(player, coord=[x, 0], dx=0, dy=1):
            return True
        # 左上から右下方向の判定
        if x == y and self.is_same(player, coord=[0, 0], dx=1, dy=1):
            return True
        # 右上から左下方向の判定
        if x + y == self.BOARD_SIZE - 1 and \
            self.is_same(player, coord=[self.BOARD_SIZE - 1, 0], dx=-1, dy=1):
            return True
    
    # どの一直線上にも配置されていない場合は、player は勝利していないので False を返す
    return False

Marubatsu.is_winner = is_winner

In [6]:
def judge(self):
    if self.move_count < self.BOARD_SIZE * 2 - 1:
        return Marubatsu.PLAYING
    # 直前に着手を行ったプレイヤーの勝利の判定
    if self.is_winner(self.last_turn):
        return self.last_turn
    # 引き分けの判定
    elif self.is_full():
        return Marubatsu.DRAW
    # 上記のどれでもなければ決着がついていない
    else:
        return Marubatsu.PLAYING      
    
Marubatsu.judge = judge

In [7]:
from collections import defaultdict
from tqdm import tqdm

def ai_match(ai, params=[{}, {}], match_num=10000, mbparams={}):
    mb = Marubatsu(**mbparams)

    # ai[0] VS ai[1] と ai[1] VS a[0] の対戦を match_num 回行い、通算成績を数える
    count_list = [ defaultdict(int), defaultdict(int)]
    for _ in tqdm(range(match_num)):
        count_list[0][mb.play(ai, params=params, verbose=False)] += 1
        count_list[1][mb.play(ai=ai[::-1], params=params[::-1], verbose=False)] += 1

    # ai[0] から見た通算成績を計算する
    count_list_ai0 = [
        # ai[0] VS ai[1] の場合の、ai[0] から見た通算成績
        { 
            "win": count_list[0][Marubatsu.CIRCLE],
            "lose": count_list[0][Marubatsu.CROSS],
            "draw": count_list[0][Marubatsu.DRAW],
        },
        # ai[1] VS ai[0] の場合の、ai[0] から見た通算成績
        { 
            "win": count_list[1][Marubatsu.CROSS],
            "lose": count_list[1][Marubatsu.CIRCLE],
            "draw": count_list[1][Marubatsu.DRAW],
        },
    ]           

    # 両方の対戦の通算成績の合計を計算する
    count_list_ai0.append({})
    for key in count_list_ai0[0]:
        count_list_ai0[2][key] = count_list_ai0[0][key] + count_list_ai0[1][key]

    # それぞれの比率を計算し、ratio_list に代入する
    ratio_list = [ {}, {}, {} ]
    for i in range(3):
        for key in count_list_ai0[i]:
            ratio_list[i][key] = count_list_ai0[i][key] / sum(count_list_ai0[i].values())
            
    # 各行の先頭に表示する文字列のリスト
    item_text_list = [ Marubatsu.CIRCLE, Marubatsu.CROSS, "total" ]    
    
    # 通算成績の回数と比率の表示
    width = max(len(str(match_num * 2)), 7)
    diff_list = [ ("count", count_list_ai0, f"{width}d"),
                  ("ratio", ratio_list, f"{width}.1%") ]
    for title, data, format in diff_list:
        print(title, end="")
        for key in data[0]:
            print(f" {key:>{width}}", end="")
        print()
        for i in range(3):
            print(f"{item_text_list[i]:5}", end="")
            for value in data[i].values():
                print(f" {value:{format}}", end="")
            print()
        print()
        
    return diff_list   

In [8]:
from ai import ai2s

ai_match(ai=[ai2s, ai2s], match_num=5000)

100%|██████████| 5000/5000 [00:01<00:00, 2895.50it/s]

count     win    lose    draw
o        2930    1425     645
x        1450    2920     630
total    4380    4345    1275

ratio     win    lose    draw
o       58.6%   28.5%   12.9%
x       29.0%   58.4%   12.6%
total   43.8%   43.5%   12.8%



[('count',
  [{'win': 2930, 'lose': 1425, 'draw': 645},
   {'win': 1450, 'lose': 2920, 'draw': 630},
   {'win': 4380, 'lose': 4345, 'draw': 1275}],
  '7d'),
 ('ratio',
  [{'win': 0.586, 'lose': 0.285, 'draw': 0.129},
   {'win': 0.29, 'lose': 0.584, 'draw': 0.126},
   {'win': 0.438, 'lose': 0.4345, 'draw': 0.1275}],
  '7.1%')]

In [9]:
ai_match(ai=[ai2s, ai2s], match_num=5000, mbparams={"count_linemark": True})

100%|██████████| 5000/5000 [00:01<00:00, 3036.09it/s]

count     win    lose    draw
o        2916    1450     634
x        1521    2846     633
total    4437    4296    1267

ratio     win    lose    draw
o       58.3%   29.0%   12.7%
x       30.4%   56.9%   12.7%
total   44.4%   43.0%   12.7%



[('count',
  [{'win': 2916, 'lose': 1450, 'draw': 634},
   {'win': 1521, 'lose': 2846, 'draw': 633},
   {'win': 4437, 'lose': 4296, 'draw': 1267}],
  '7d'),
 ('ratio',
  [{'win': 0.5832, 'lose': 0.29, 'draw': 0.1268},
   {'win': 0.3042, 'lose': 0.5692, 'draw': 0.1266},
   {'win': 0.4437, 'lose': 0.4296, 'draw': 0.1267}],
  '7.1%')]

In [10]:
a = False

In [11]:
%%timeit

if a:
    pass

16.2 ns ± 0.347 ns per loop (mean ± std. dev. of 7 runs, 100,000,000 loops each)


In [12]:
from ai import ai14s

ai_match(ai=[ai14s, ai14s], match_num=5000)

100%|██████████| 5000/5000 [00:11<00:00, 441.49it/s]

count     win    lose    draw
o           0       0    5000
x           0       0    5000
total       0       0   10000

ratio     win    lose    draw
o        0.0%    0.0%  100.0%
x        0.0%    0.0%  100.0%
total    0.0%    0.0%  100.0%



[('count',
  [{'win': 0, 'lose': 0, 'draw': 5000},
   {'win': 0, 'lose': 0, 'draw': 5000},
   {'win': 0, 'lose': 0, 'draw': 10000}],
  '7d'),
 ('ratio',
  [{'win': 0.0, 'lose': 0.0, 'draw': 1.0},
   {'win': 0.0, 'lose': 0.0, 'draw': 1.0},
   {'win': 0.0, 'lose': 0.0, 'draw': 1.0}],
  '7.1%')]

In [13]:
ai_match(ai=[ai14s, ai14s], match_num=5000, mbparams={"count_linemark": True})

100%|██████████| 5000/5000 [00:11<00:00, 450.12it/s]

count     win    lose    draw
o           0       0    5000
x           0       0    5000
total       0       0   10000

ratio     win    lose    draw
o        0.0%    0.0%  100.0%
x        0.0%    0.0%  100.0%
total    0.0%    0.0%  100.0%



[('count',
  [{'win': 0, 'lose': 0, 'draw': 5000},
   {'win': 0, 'lose': 0, 'draw': 5000},
   {'win': 0, 'lose': 0, 'draw': 10000}],
  '7d'),
 ('ratio',
  [{'win': 0.0, 'lose': 0.0, 'draw': 1.0},
   {'win': 0.0, 'lose': 0.0, 'draw': 1.0},
   {'win': 0.0, 'lose': 0.0, 'draw': 1.0}],
  '7.1%')]

In [14]:
from marubatsu import Markpat

mb = Marubatsu(count_linemark=True)
mb.move(0, 0)
print(mb)
circlenum = mb.rowcount[Marubatsu.CIRCLE][0]
crossnum = mb.rowcount[Marubatsu.CROSS][0]
emptynum = mb.BOARD_SIZE - circlenum - crossnum
print(Markpat(circlenum, crossnum, emptynum))


Turn x
O..
...
...

Markpat(last_turn=1, turn=0, empty=2)


In [15]:
circlenum = mb.rowcount[Marubatsu.CIRCLE][1]
crossnum = mb.rowcount[Marubatsu.CROSS][1]
emptynum = mb.BOARD_SIZE - circlenum - crossnum
print(Markpat(circlenum, crossnum, emptynum))
circlenum = mb.rowcount[Marubatsu.CIRCLE][2]
crossnum = mb.rowcount[Marubatsu.CROSS][2]
emptynum = mb.BOARD_SIZE - circlenum - crossnum
print(Markpat(circlenum, crossnum, emptynum))

Markpat(last_turn=0, turn=0, empty=3)
Markpat(last_turn=0, turn=0, empty=3)


In [16]:
for y in range(mb.BOARD_SIZE):
    circlenum = mb.rowcount[Marubatsu.CIRCLE][y]
    crossnum = mb.rowcount[Marubatsu.CROSS][y]
    emptynum = mb.BOARD_SIZE - circlenum - crossnum
    print(Markpat(circlenum, crossnum, emptynum))

Markpat(last_turn=1, turn=0, empty=2)
Markpat(last_turn=0, turn=0, empty=3)
Markpat(last_turn=0, turn=0, empty=3)


In [17]:
lista = [1, 2, 3]
listb = [4, 5, 6]
for a, b in zip(lista, listb):
    print(a + b)

5
7
9


In [18]:
for circlenum, crossnum in zip(mb.rowcount[Marubatsu.CIRCLE], mb.rowcount[Marubatsu.CROSS]):
    emptynum = mb.BOARD_SIZE - circlenum - crossnum
    print(Markpat(circlenum, crossnum, emptynum))

Markpat(last_turn=1, turn=0, empty=2)
Markpat(last_turn=0, turn=0, empty=3)
Markpat(last_turn=0, turn=0, empty=3)


In [19]:
from collections import defaultdict

def count_markpats(self):
    markpats = defaultdict(int)
    
    if self.count_linemark:
        for countdict in [self.rowcount, self.colcount, self.diacount]:
            for circlecount, crosscount in zip(countdict[Marubatsu.CIRCLE], countdict[Marubatsu.CROSS]):
                emptycount = self.BOARD_SIZE - circlecount - crosscount
                if self.last_turn == Marubatsu.CIRCLE:
                    markpats[(circlecount, crosscount, emptycount)] += 1
                else:
                    markpats[(crosscount, circlecount, emptycount)] += 1
    else:
        # 横方向と縦方向の判定
        for i in range(self.BOARD_SIZE):
            count = self.count_marks(coord=[0, i], dx=1, dy=0, datatype="tuple")
            markpats[count] += 1
            count = self.count_marks(coord=[i, 0], dx=0, dy=1, datatype="tuple")
            markpats[count] += 1
        # 左上から右下方向の判定
        count = self.count_marks(coord=[0, 0], dx=1, dy=1, datatype="tuple")
        markpats[count] += 1
        # 右上から左下方向の判定
        count = self.count_marks(coord=[2, 0], dx=-1, dy=1, datatype="tuple")
        markpats[count] += 1

    return markpats   

Marubatsu.count_markpats = count_markpats

In [20]:
ai_match(ai=[ai14s, ai14s], match_num=5000)

100%|██████████| 5000/5000 [00:10<00:00, 461.80it/s]

count     win    lose    draw
o           0       0    5000
x           0       0    5000
total       0       0   10000

ratio     win    lose    draw
o        0.0%    0.0%  100.0%
x        0.0%    0.0%  100.0%
total    0.0%    0.0%  100.0%



[('count',
  [{'win': 0, 'lose': 0, 'draw': 5000},
   {'win': 0, 'lose': 0, 'draw': 5000},
   {'win': 0, 'lose': 0, 'draw': 10000}],
  '7d'),
 ('ratio',
  [{'win': 0.0, 'lose': 0.0, 'draw': 1.0},
   {'win': 0.0, 'lose': 0.0, 'draw': 1.0},
   {'win': 0.0, 'lose': 0.0, 'draw': 1.0}],
  '7.1%')]

In [21]:
ai_match(ai=[ai14s, ai14s], match_num=5000, mbparams={"count_linemark": True})

100%|██████████| 5000/5000 [00:05<00:00, 887.25it/s]

count     win    lose    draw
o           0       0    5000
x           0       0    5000
total       0       0   10000

ratio     win    lose    draw
o        0.0%    0.0%  100.0%
x        0.0%    0.0%  100.0%
total    0.0%    0.0%  100.0%



[('count',
  [{'win': 0, 'lose': 0, 'draw': 5000},
   {'win': 0, 'lose': 0, 'draw': 5000},
   {'win': 0, 'lose': 0, 'draw': 10000}],
  '7d'),
 ('ratio',
  [{'win': 0.0, 'lose': 0.0, 'draw': 1.0},
   {'win': 0.0, 'lose': 0.0, 'draw': 1.0},
   {'win': 0.0, 'lose': 0.0, 'draw': 1.0}],
  '7.1%')]